In [2]:
import torch
import inspect
import torch.optim as optim
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.modules.activation as A
from tqdm import tqdm
%pylab inline
import torchvision
import torchvision.transforms as transforms
from livelossplot import PlotLosses
import os, errno
from torch.distributions.multivariate_normal import MultivariateNormal

Populating the interactive namespace from numpy and matplotlib


In [18]:
mnist_data = torchvision.datasets.MNIST('../datasets/mnist', download=True, train=True,
                                           transform=transforms.Compose([
                           transforms.ToTensor(),
#                            transforms.Normalize((0.1307,), (0.3081,))
                       ]))
mnist_loader = torch.utils.data.DataLoader(mnist_data, batch_size=32, shuffle=True)

In [4]:
# list(mnist_loader)[0][0][0]

In [22]:
class Encoder(nn.Module): 
    def __init__(self, nz=100):
        """
        Encoder(z|X) = Q(z|X) = N(z|mu(X;theta), Sigma(X;theta))
        """
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, nz)
        self.fc22 = nn.Linear(400, nz)
        
    def forward(self, X):
        """
        Input 
        - X: datapoint
        
        Output
        - mu_z(X): means. Shape: [nz, 1]
        - Sigma_z(X): variances. Shape: [nz, 1]
        """
        X = self.fc1(X)
        X = F.relu(X)
        return self.fc21(X), self.fc22(X)
     
class Decoder(nn.Module):
    def __init__(self, nz=100):
        """
        P(X|z;theta) = N(X|f(z;theta), sigma^2 * I)
        """
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(nz, 400)
        self.fc2 = nn.Linear(400, 784)
        
    def forward(self, z):
        """
        Input
        - z: noise
        
        Output:
        - mu_X(z): mean of gaussian X. Shape: [flatten(X), 1]
        """
        z = self.fc1(z)
        z = F.relu(z)
        z = self.fc2(z)
        return F.sigmoid(z)

In [32]:
Q = Encoder(nz=20)
D = Decoder(nz=20)

# def KL_divergence_gaussians(noise_mus, variance):
#     """
#     From Appendix B: https://arxiv.org/pdf/1312.6114.pdf
#     """    
# #     Q_z_X = torch.distributions.multivariate_normal.MultivariateNormal(
# #     loc, covariance_matrix=None)
# #     P_z = torch.distributions.multivariate_normal.MultivariateNormal(
# #     torch.zeros(), covariance_matrix=torch.eye())
# #     Q_loss = torch.distributions.kl.kl_divergence(Q_z_X, P_z)

#     return .5 * torch.sum(1 + torch.log(variance) - noise_mus**2 - variance)

# def sample_noise(mu, variance):
#     # Sample noise from Q(z|X) in a differentiable way using the reparameterization trick
#     eps = torch.randn_like(mu)
#     prod = variance**(0.5) * eps
#     return prod+z_mus

def KL_divergence_gaussians(mu, logvar):
    return .5 * torch.sum(1 + logvar - mu**2 - logvar.exp())

def sample_noise(mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

In [34]:
# fixed_noise = torch.randn([32, 20, 1, 1])
fixed_noise = torch.randn([32, 20])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
opt = optim.Adam(list(Q.parameters()) + list(D.parameters()), lr=1e-3)
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir="mnist")
for epoch in range(5):
    for batch_idx, X in tqdm(enumerate(mnist_loader)):
        X = X[0]
        X = X.to(device, dtype=torch.float32)
        batch_size = X.shape[0]
    
        X = X.view(-1, 784)
        z_mus, z_sigmas = Q(X)
        
        z = sample_noise(z_mus, z_sigmas) # Should be 32x100... 
        
        Q.zero_grad()       
        KLD = KL_divergence_gaussians(z_mus, z_sigmas)

        D.zero_grad()
        BCE = F.binary_cross_entropy(D(z), X, reduction='sum')
        print("BCE " + str(BCE))
        print("KLD " + str(-KLD))
        loss = BCE - KLD
        loss.backward()
        writer.add_scalar('loss', loss, global_step=(epoch+1)*batch_idx)

        opt.step()
        if batch_idx % 500 == 0:
            import torchvision.utils as vutils
            gen_img = D(fixed_noise).view(-1, 1, 28, 28)
            vutils.save_image(gen_img.detach(), F"images/fake_samples-{epoch}-{batch_idx}.png", normalize=True)
            vutils.save_image(X.view(-1, 1, 28, 28), F"images/true_samples-{epoch}-{batch_idx}.png", normalize=True)
#             import ipdb; ipdb.set_trace()
            vutils.save_image(D(Q(X)[0]).view(-1, 1, 28, 28) , F"images/recon_samples-{epoch}-{batch_idx}.png", normalize=True)






0it [00:00, ?it/s]/Users/richard/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")





4it [00:00, 36.00it/s]

BCE tensor(3070.7207, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(778.6455, grad_fn=<NegBackward>)
BCE tensor(3145.0376, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(768.8815, grad_fn=<NegBackward>)
BCE tensor(2666.4932, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(722.7473, grad_fn=<NegBackward>)
BCE tensor(2927.4917, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(711.4383, grad_fn=<NegBackward>)
BCE tensor(2955.6836, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(781.0661, grad_fn=<NegBackward>)
BCE tensor(3029.4609, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(715.4710, grad_fn=<NegBackward>)
BCE tensor(2773.7605, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(739.3893, grad_fn=<NegBackward>)
BCE tensor(2961.1350, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(746.0975, grad_fn=<NegBackward>)
BCE tensor(2714.0540, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(730.6452, grad_fn=<NegBackward>)







9it [00:00, 42.56it/s]




14it [00:00, 44.14it/s]

BCE tensor(2988.0840, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(758.0694, grad_fn=<NegBackward>)
BCE tensor(2791.7673, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(753.0459, grad_fn=<NegBackward>)
BCE tensor(2969.3586, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(744.1324, grad_fn=<NegBackward>)
BCE tensor(2627.7095, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(747.7255, grad_fn=<NegBackward>)
BCE tensor(2981.5964, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(772.6428, grad_fn=<NegBackward>)
BCE tensor(3017.4116, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(803.4797, grad_fn=<NegBackward>)
BCE tensor(2877.5403, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(804.2906, grad_fn=<NegBackward>)
BCE tensor(2924.2454, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(809.5558, grad_fn=<NegBackward>)
BCE tensor(2655.9478, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(762.7988, grad_fn=<NegBackward>)
BCE tensor(3095.9099, grad_fn=<BinaryCrossEntropyBackward>)
KLD 






20it [00:00, 46.34it/s]




25it [00:00, 46.19it/s]

...

BCE tensor(2858.1555, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(838.4366, grad_fn=<NegBackward>)
BCE tensor(2606.2488, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(784.4173, grad_fn=<NegBackward>)
BCE tensor(2801.3306, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(844.5826, grad_fn=<NegBackward>)
BCE tensor(2583.6519, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(864.6423, grad_fn=<NegBackward>)
BCE tensor(2535.1821, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(871.2173, grad_fn=<NegBackward>)
BCE tensor(2541.3608, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(819.4092, grad_fn=<NegBackward>)
BCE tensor(2627.5439, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(873.4930, grad_fn=<NegBackward>)
BCE tensor(2831.7703, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(858.5235, grad_fn=<NegBackward>)
BCE tensor(2582.6550, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(796.5409, grad_fn=<NegBackward>)
BCE tensor(2425.0415, grad_fn=<BinaryCrossEntropyBackward>)
KLD 






1867it [00:42, 43.64it/s]




1872it [00:42, 43.64it/s]

BCE tensor(2458.2578, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(787.7841, grad_fn=<NegBackward>)
BCE tensor(2841.5444, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(809.5728, grad_fn=<NegBackward>)
BCE tensor(2706.9939, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(808.3735, grad_fn=<NegBackward>)
BCE tensor(2433.9390, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(788.4813, grad_fn=<NegBackward>)
BCE tensor(2505.4448, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(764.9125, grad_fn=<NegBackward>)
BCE tensor(2441.0591, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(727.7825, grad_fn=<NegBackward>)
BCE tensor(2511.5022, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(731.3463, grad_fn=<NegBackward>)
BCE tensor(2659.7354, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(783.5833, grad_fn=<NegBackward>)
BCE tensor(2440.4351, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(742.1822, grad_fn=<NegBackward>)







1875it [00:42, 43.61it/s]

BCE tensor(2503.7808, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(792.7695, grad_fn=<NegBackward>)
BCE tensor(2565.2512, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(800.7151, grad_fn=<NegBackward>)
BCE tensor(2572.7170, grad_fn=<BinaryCrossEntropyBackward>)
KLD tensor(804.6165, grad_fn=<NegBackward>)


# Notes
- Make sure the BCE loss is "SUM" reduction. MSE doesn't train... Just because of weighting?
- Using logvar instead of variance for network outputs fixes nan errors (over/underflow?) in the KLD loss